In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import os

# Value Investing

Value investing is an investment strategy that involves picking stocks that appear to be trading for less than their intrinsic or book value. Value investors actively ferret out stocks they think the stock market is underestimating. ([Source: Investopedia](https://www.investopedia.com/terms/v/valueinvesting.asp))

In [2]:
# change the directiory to root to allow importing of py files saved in algorithmictrading
os.chdir(r'..\\')
from algorithmictrading.secrets import IEX_CLOUD_API_TOKEN

# Data Loading - S&P 500 Index
The S&P 500 Index is one of the most common benchmarks for US Large Cap stocks. It tracks the performance of 500 of the largest companies in the United States.

You can substitute any list of tickers for this equal weight walk-through. The list of stocks should be aved in the `\data` folder.

In [3]:
stocks = pd.read_csv(r'.\data\sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


# Connecting to the IEX API
We will be using the free IEX Cloud API for the market data. Data is purposefully scrambled and is NOT meant for production!

[Documentation can be found here.](https://iexcloud.io/docs/api/#testing-sandbox)

We can use the base URL and concatenate a string from the API IEX documentation in order to pull the data.
We can pass the following into the string for a specific data request:
- `symbol`
- `token`

In [4]:
BASE_URL = 'https://sandbox.iexapis.com/stable'
symbol = 'AAPL'
stats = f'/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(BASE_URL+stats).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2273068085290,
 'week52high': 140.16,
 'week52low': 58.11,
 'week52change': 0.6930002855204533,
 'sharesOutstanding': 17627951803,
 'float': 0,
 'avg10Volume': 119952493,
 'avg30Volume': 119251118,
 'day200MovingAvg': 117.41,
 'day50MovingAvg': 129.88,
 'employees': 0,
 'ttmEPS': 3.28,
 'ttmDividendRate': 0.8109606033446924,
 'dividendYield': 0.006527221989413817,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-03',
 'nextEarningsDate': '0',
 'peRatio': 37.7532014328182,
 'beta': 1.187940935973023,
 'maxChangePercent': 48.59744799393545,
 'year5ChangePercent': 4.737394295947578,
 'year2ChangePercent': 2.5684902028449477,
 'year1ChangePercent': 0.6775859241606804,
 'ytdChangePercent': -0.030486158533369237,
 'month6ChangePercent': 0.357453117060417,
 'month3ChangePercent': 0.0389479179992847,
 'month1ChangePercent': 0.05310935926756605,
 'day30ChangePercent': 0.05234547912397358,
 'day5ChangePercent': -0.01694284548365485}

In [5]:
data['peRatio']

37.7532014328182

# Making Batch API Calls
Making a single http request is really slow. We are much better served breaking up our security list into small batches. The IEX API limits 100 symbols per batch, so we we will make 6 http requests.

For our first **value** factor, will will use:
- `peRatio` - is the ratio for valuing a company that measures its current share price relative to its per-share earnings; also sometimes referred to as price multiple or the earnings multiple; it's used by investors and analysts to determine the relative value of a company's shares in an apples-to-apples comparison ([source: Investopedia](https://www.investopedia.com/terms/p/price-earningsratio.asp))


In [6]:
def make_chunks(df):
     return np.array_split(df['Ticker'].to_list(), np.ceil(len(df) / 100))

def get_data_batch(df):
    df_list = []
    chunks = make_chunks(df)
    for chunk in chunks:
        ticker_strings = ','.join(chunk)
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={ticker_strings}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        tickers = [k for k in data.keys()]
        latestprices = [data[k]['quote']['latestPrice'] for k in data.keys()]
        pe_ratios = [data[k]['quote']['peRatio'] for k in data.keys()]
        df = pd.DataFrame({'ticker': tickers, 'latest_price': latestprices, 'pe_ratio': pe_ratios})
        df_list.append(df)
    return  pd.concat(df_list, ignore_index=True)

In [7]:
value_df = get_data_batch(stocks)
value_df.head()

,ticker,latest_price,pe_ratio
0,A,127.08,55.59
1,AAL,15.70,-1.13
2,AAP,173.49,25.00
3,AAPL,135.38,41.88
4,ABBV,114.23,25.14


# Filtering for Relative Value and Calculating Share Amounts
The next step is to drop the securities that are "over-priced" (they have high a p/e ratio)
- `sort_values()` by `pe_ratio`
- take the top 50 stocks
- Calculate the shares needed to buy, given a portfolio amount (assuming equal weighting)

In [17]:
def transform_value_df(df, stock_cutoff=50):
    df = df.copy()
    return (df.sort_values('pe_ratio')
              .loc[df['pe_ratio'] > 0] # filter out companies with negative earnings
              .reset_index(drop=True)
              .iloc[:stock_cutoff]
           )

def get_share_amounts(df, portfolio_size=50000000):
    share_amounts = portfolio_size / len(df.index)
    return df.assign(recommended_trades= lambda x: np.floor(share_amounts /  x['latest_price']))

In [18]:
final_df = (transform_value_df(value_df)
            .pipe(get_share_amounts)
           )
final_df.head()

,ticker,latest_price,pe_ratio,recommended_trades
0,NRG,41.800,2.53,23923.0
1,NLOK,21.791,4.17,45890.0
2,AIV,4.920,4.88,203252.0
3,UNM,24.880,5.30,40192.0
4,BIO,613.230,5.45,1630.0


# Improving On Our Value Strategy
We will introduce additional ways to infer a company's intrinsic vaulation.

The following factors will be extraced from IEX CLOUD:
- `priceToBook`
    - Is calculated by dividing the company's stock price per share by its book value per share (BVPS). An asset's book value is equal to its carrying value on the balance sheet, and companies calculate it netting the asset against its accumulated depreciation
- `priceToSales`
    - Is a valuation ratio that compares a company’s stock price to its revenues. It is an indicator of the value that financial markets have placed on each dollar of a company’s sales or revenues
    - The P/S ratio can be calculated either by dividing the company’s market capitalization by its total sales over a designated period (usually twelve months) or on a per-share basis by dividing the stock price by sales per share. The P/S ratio is also known as a sales multiple or revenue multiple
- `enterpriseValue`
    - Is a measure of a company's total value, often used as a more comprehensive alternative to equity market capitalization
    - EV includes in its calculation the market capitalization of a company but also short-term and long-term debt as well as any cash on the company's balance sheet
    - We will use this in conjuction with `grossProfit` and `EBITDA` to calculate two additional ratios
- `EBITDA` (earnings before interest, taxes, depreciation, and amortization)
    - Is a measure of a company's overall financial performance and is used as an alternative to net income in some circumstances
    - We will use this in conjuction with `enterpriseValue` to calculate a ratio
- `grossProfit`
    - Is the profit a company makes after deducting the costs associated with making and selling its products, or the costs associated with providing its services
    - Gross profit will appear on a company's income statement and can be calculated by subtracting the cost of goods sold (COGS) from revenue (sales)
    - We will use this in conjuction with `enterpriseValue` to calculate a ratio
- `enterpriseValue/EBITDA` (derived ratio)
    -  Is a financial valuation ratio that measures a company's return on investment (ROI)
    - The EBITDA/EV ratio may be preferred over other measures of return because it is normalized for differences between companies
- `enterpriseValue/grossProfit` (derived ratio)
    - Is a measure of the value of a stock that compares a company's enterprise value to its revenue
    
 [Source: Investopedia](https://www.investopedia.com/)

In [44]:
def value_df_transform(df):
    return (df.copy()
              .assign(enterpriseValue_EBITDA= lambda x: x['enterpriseValue'] / x['EBITDA'],
                      enterpriseValue_grossProfit= lambda x: x['enterpriseValue'] / x['grossProfit'])
              .drop(columns=['enterpriseValue', 'EBITDA', 'grossProfit'])
              .rename(columns={'enterpriseValue_EBITDA': 'enterpriseValue/EBITDA',
                               'enterpriseValue_grossProfit' : 'enterpriseValue/grossProfit'})
           )

def get_value_data_batch(df):
    df_list = []
    chunks = make_chunks(df)
    for chunk in chunks:
        ticker_strings = ','.join(chunk)
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote,advanced-stats&symbols={ticker_strings}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        
        # get variables
        tickers = [k for k in data.keys()]
        latestprices = [data[k]['quote']['latestPrice'] for k in data.keys()]
        pe_ratios = [data[k]['quote']['peRatio'] for k in data.keys()]
        price_books = [data[k]['advanced-stats']['priceToBook'] for k in data.keys()]
        price_sales = [data[k]['advanced-stats']['priceToSales'] for k in data.keys()]
        enterpriseValues = [data[k]['advanced-stats']['enterpriseValue'] for k in data.keys()]
        ebitas = [data[k]['advanced-stats']['EBITDA'] for k in data.keys()]
        grossprofits = [data[k]['advanced-stats']['grossProfit'] for k in data.keys()]
        df = (pd.DataFrame({'ticker': tickers, 
                           'latest_price': latestprices, 
                           'peRatio': pe_ratios,
                           'priceToBook': price_books,
                           'priceToSales': price_sales,
                           'enterpriseValue': enterpriseValues,
                           'EBITDA': ebitas,
                           'grossProfit': grossprofits
                          })
              .pipe(value_df_transform)
             )
        df_list.append(df)
    return  pd.concat(df_list, ignore_index=True)

In [45]:
val_df2 = get_value_data_batch(stocks)
val_df2.head()

,ticker,latest_price,peRatio,priceToBook,priceToSales,enterpriseValue/EBITDA,enterpriseValue/grossProfit
0,A,127.890,55.11,8.00,7.5000,34.617412,14.312675
1,AAL,16.680,-1.22,-1.80,0.3916,-6.043717,1.413617
2,AAP,169.942,24.89,3.20,1.1900,11.667066,2.651611
3,AAPL,135.200,39.84,34.70,8.4100,28.539458,21.579216
4,ABBV,117.490,25.10,12.88,4.8500,16.568837,9.581256


# Solving for Missing Data
Some values are not available - we can `fillna()` using the column average by leveraging `apply()` and `lambda` functionality.

In [46]:
# missing data
val_df2.loc[val_df2.isna().any(axis='columns')]

,ticker,latest_price,peRatio,priceToBook,priceToSales,enterpriseValue/EBITDA,enterpriseValue/grossProfit
40,AON,208.740,NaN,NaN,NaN,NaN,NaN
71,BRK.B,242.840,NaN,NaN,NaN,NaN,NaN
118,CTL,11.000,9.95,NaN,NaN,NaN,NaN
136,DISCK,34.405,NaN,NaN,NaN,NaN,NaN
165,ETFC,49.710,14.37,NaN,NaN,NaN,NaN
168,EVRG,54.920,NaN,NaN,NaN,NaN,NaN
190,FOX,31.659,NaN,NaN,NaN,NaN,NaN
192,FRC,162.760,NaN,NaN,NaN,NaN,NaN
204,GOOG,1790.590,NaN,NaN,NaN,NaN,NaN
326,MYL,16.450,32.20,NaN,NaN,NaN,NaN


In [47]:
def fill_missing_vals(df):
    df = df.copy()
    return (df.set_index('ticker')
              .apply(lambda x: x.fillna(x.mean()))
              .reset_index()
           )

In [48]:
val_df2_fill = fill_missing_vals(val_df2)
val_df2_fill.loc[val_df2.isna().any(axis='columns')]

,ticker,latest_price,peRatio,priceToBook,priceToSales,enterpriseValue/EBITDA,enterpriseValue/grossProfit
40,AON,208.740,-0.843354,9.528179,4.697346,74.302827,8.719675
71,BRK.B,242.840,-0.843354,9.528179,4.697346,74.302827,8.719675
118,CTL,11.000,9.950000,9.528179,4.697346,74.302827,8.719675
136,DISCK,34.405,-0.843354,9.528179,4.697346,74.302827,8.719675
165,ETFC,49.710,14.370000,9.528179,4.697346,74.302827,8.719675
168,EVRG,54.920,-0.843354,9.528179,4.697346,74.302827,8.719675
190,FOX,31.659,-0.843354,9.528179,4.697346,74.302827,8.719675
192,FRC,162.760,-0.843354,9.528179,4.697346,74.302827,8.719675
204,GOOG,1790.590,-0.843354,9.528179,4.697346,74.302827,8.719675
326,MYL,16.450,32.200000,9.528179,4.697346,74.302827,8.719675


# Calculating Robust Value Scores
- Normalize rankings by converting to percentile `rank(pct=True)`
- Sorts for the lowest scoring stocks
- Take the top 50 to create our stock list
- Apply the `generate_robust_value` function introduced earlier to get the exact share amounts needed, given a portfolio size

In [49]:
def generate_robust_value_score(df, stock_cutoff=50):
    return (df.copy()
              .set_index(['ticker', 'latest_price'])
              .rank(pct=True)
              .assign(rv_score= lambda x: x.mean(axis='columns'))
              .sort_values('rv_score')
              .reset_index()
              .head(stock_cutoff)
           )

In [52]:
final_df2 = (generate_robust_value_score(val_df2_fill)
             .pipe(get_share_amounts)
            )
final_df2.head()

,ticker,latest_price,peRatio,priceToBook,priceToSales,enterpriseValue/EBITDA,enterpriseValue/grossProfit,rv_score,recommended_trades
0,F,10.372,0.023762,0.111881,0.009901,0.039604,0.095050,0.056040,96413.0
1,AIG,42.729,0.112871,0.057426,0.069307,0.045545,0.015842,0.060198,23403.0
2,AAL,16.680,0.154455,0.047525,0.023762,0.029703,0.053465,0.061782,59952.0
3,UNM,25.820,0.196040,0.053465,0.031683,0.035644,0.003960,0.064158,38729.0
4,MET,52.070,0.207921,0.061386,0.057426,0.047525,0.011881,0.077228,19204.0


# Exporting Data to Excel

Pandas can easily output to a csv file of xlsx file natively. However, if we want to output to a styled xlsx file, we can use `xlsxwriter` to customize the output to a much greater degree. 

In [65]:
writer = pd.ExcelWriter(r'.\data\value_recommended_trades.xlsx', engine='xlsxwriter')
final_df2.to_excel(writer, sheet_name='Recommended Trades', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

dollar_format = writer.book.add_format(
        {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

pct_format = writer.book.add_format(
        {
        'num_format':'0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

float_format = writer.book.add_format(
        {
        'num_format':'0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

integer_format = writer.book.add_format(
        {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )


column_formats = { 
                    'A': ['ticker', string_format],
                    'B': ['latest_price', dollar_format],
                    'C': ['peRatio', pct_format],
                    'D': ['priceToBook', pct_format],
                    'E': ['priceToSales', pct_format],
                    'F': ['enterpriseValue/EBITDA', pct_format],
                    'G': ['enterpriseValue/grossProfit', pct_format],
                    'H': ['rv_score', float_format],
                    'I': ['recommended_trades', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
    
writer.sheets['Recommended Trades'].hide_gridlines(2)

writer.save()